In [2]:
# import pysqlite3 as sqlite3
# print(sqlite3.sqlite_version)
import pysqlite3
print(pysqlite3.sqlite_version)

3.46.1


In [4]:
import pysqlite3
import sys

# Override built-in sqlite3 module
sys.modules['sqlite3'] = pysqlite3

import chromadb

In [ ]:
# local storage chroma example
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="mails")


In [1]:
import os 
# from data_loader.mail  import  Email
mails = []
DIR = "/home/misia/Programowanie/hackathonAI/maind-controllers/data/mail-dataset"
for f  in os.listdir(DIR):
    m = "".join(open(DIR+"/"+f, "r").readlines())
    mails.append(m)
    
mails[0]

'Od: Michał Wróbel <michal.wrobel@szpilex.ai>\nWysłano: 2025-02-18 09:07\nTo: Natalia Król <natalia.krol@ecom-client.com>, Bartosz Maj <bartosz.maj@szpilex.ai>\nTemat: Hybrydowy asystent – zasady routingu\n\nCześć,\nw projekcie hybrydowym zakładamy:\n- voicebot obsługuje część rozmowy (identyfikacja, temat),\n- RAG generuje podpowiedzi dla konsultanta na ekranie,\n- w niektórych przypadkach voicebot czyta odpowiedź RAG klientowi.\n\nPotrzebujemy tabeli: które intencje idą pełnym "self-service", a które tylko jako podpowiedź dla agenta.\n\nMichał\n\n--\nMichał Wróbel\nProduct Manager – CX Automation | SZPILEX.ai\n\n\nOd: Natalia Król <natalia.krol@ecom-client.com>\nWysłano: 2025-02-18 10:36\nDo: Michał Wróbel <michal.wrobel@szpilex.ai>\nTemat: Hybrydowy asystent – zasady routingu\n\nPropozycja:\n- self-service: status zamówienia, godziny otwarcia, proste FAQ,\n- tylko podpowiedź: reklamacje, problemy z płatnościami, opóźnienia dostawy,\n- obowiązkowo do agenta: groźby pozwu, skargi form

In [2]:
from backend.ai_core.data_loader.load_data import process_mail
from backend.ai_core.graphs import *
from tqdm import tqdm

from backend.schemas.mail import EmailAnalysisSchema

    
def load_emails(mails: list[str]) -> list:
    emails = []
    for m in tqdm(mails):
        email = process_mail(m)
        emails.append(email)
    return emails

def run(emails: list[EmailAnalysisSchema]):
    dag_connections = build_dag(emails)
    rolling_states = calculate_rolling_states(emails, dag_connections)
    topic_tags = assign_topic_tags(emails, dag_connections)
    
    print("DAG Connections:")
    for c in dag_connections:
        print(c)
        print()
    print("\nRolling States:")
    for e in emails:
        print(f"Email: {e.topic}")
        print(f"Rolling State: {rolling_states.get(e, 'No State')}")
        print("-----")

    print("\nTopic Tags:")
    for email, topics in topic_tags.items():
        print(f"Email: {email.topic}")
        print(f"Topics: {', '.join(topics)}")
        print("-----")
 

emails = load_emails(mails)
# run(emails)

100%|██████████| 97/97 [10:42<00:00,  6.62s/it]


In [8]:
collection = chroma_client.create_collection(name="test_mails")
collection2 = chroma_client.create_collection(name="test_summary_mails")

In [74]:
chroma_client.get_or_create_collection(name="mails")

Collection(name=mails)

In [75]:
# import chromadb

# chroma = chromadb.HttpClient(
#     host="localhost",
#     port=8003
# )

In [ ]:
collection.add(
    ids=[f"id{i}" for i in range(len(emails))],
    documents=[e.extra['data'] for e in emails]
)
collection2.add(
    ids=[f"id{i}" for i in range(len(emails))],
    documents=[e.summary  for e  in emails]
)


In [67]:
import importlib
import backend.ai_core.llm_api.api

importlib.reload(backend.ai_core.llm_api.api)


<module 'backend.ai_core.llm_api.api' from '/home/misia/Programowanie/hackathonAI/maind-controllers/backend/ai_core/llm_api/api.py'>

In [68]:
from backend.ai_core.llm_api.api import get_rag_response


response = get_rag_response("Kiedy jest organizowany bieg chartatywny?", 
                 collection,
                 5, 
                 2, )
response 

('2025-05-11',
 'Temat: Firmowa drużyna na Bieg Charytatywny “IT Run for Kids”\nOd: CSR Committee SZPILEX.ai <csr@szpilex.ai>\nWysłano: 2025-04-06 13:40\nDo: wszyscy_pracownicy@szpilex.ai\n\nCześć,\n\nzbieramy zapisy do firmowej drużyny na:\n\nBieg Charytatywny “IT Run for Kids”\nData: 2025-05-11 (niedziela)\nDystans: 5 km (bieg) + 5 km (marsz nordic walking)\nMiejsce: Warszawa, okolice PGE Narodowego\n\nCel:\n- wsparcie fundacji finansującej zajęcia z programowania dla dzieci z domów dziecka.\n\nSzczegóły:\n- opłatę startową pokrywa firma,\n- uczestnicy dostaną:\n  - koszulkę techniczną z logo SZPILEX.ai,\n  - pakiet startowy od organizatorów.\n\nZapisy:\n- do 2025-04-25,\n- formularz w intranecie → kafelek “IT Run for Kids”.\n\nZapraszamy zarówno biegaczy, jak i osoby, które po prostu chcą przejść dystans spacerem :)\n\nCSR Committee\nSZPILEX.ai\n\n\nOd: Kinga Wójcik <kinga.wojcik@shiftplanner.io>\nWysłano: 2025-02-12 09:00\nDo: Natalia Paź <natalia.paz@fashionretail.eu>\nDW: Dział O

In [76]:
from backend.ai_core.llm_api.api import retirve_context_data


ids = retirve_context_data(mails[2], collection, 15, 2 )
mails_to_run = [emails[2]] + [ e for i, e in enumerate(emails) if f"id{i}" in ids]
run(mails_to_run)

Calculating rolling states...
Assigning topic tags...
DAG Connections:

Rolling States:
Email: [Szkolenie] Skuteczne prezentacje dla PM-ów (online)
Rolling State: Stan aktualny: Planowanie
Zadania: Zapisy na szkolenie do 2025-04-10
Decyzje: Szkolenie online 2025-04-17, 14:00-16:30 dla zespołu Product Management
-----

Topic Tags:
Email: [Szkolenie] Skuteczne prezentacje dla PM-ów (online)
Topics: Product Management Training, Online Webinar, PM Development
-----


In [85]:
id_src = 1
ids = retirve_context_data(mails[id_src], collection, 15, 5 )
mails_to_run = [emails[id_src]] + [ e for i, e in enumerate(emails) if f"id{i}" in ids]
run(mails_to_run)

Calculating rolling states...
Assigning topic tags...
DAG Connections:

Rolling States:
Email: DonorLink – segmentacja i scoring darczyńców
Rolling State: Stan: Planowanie
Zadania: Julia Kaczmarczyk - aktualizuje system segmentacji i scoringu darczyńców
Decyzje: scoring jako wskazówka, możliwość ręcznej zmiany segmentu, ograniczenia kampanii dla darczyńców z niską aktywnością
-----

Topic Tags:
Email: DonorLink – segmentacja i scoring darczyńców
Topics: DonorLink, Donor Segmentation
-----


In [87]:
id_src = 3
ids = retirve_context_data(mails[id_src], collection, 15, 5 )
mails_to_run = [emails[id_src]] + [ e for i, e in enumerate(emails) if f"id{i}" in ids]
run(mails_to_run)

Calculating rolling states...
Assigning topic tags...
DAG Connections:

Rolling States:
Email: System floty – model przeglądów i alertów serwisowych
Rolling State: Status: Planowanie
Aktywne zadania: Agata Kowal opracowuje model systemu floty, Dział Serwisu Floty i Joanna Baran dostarczają wymagania
Kluczowe decyzje:Implementacja systemu przeglądów i alertów serwisowych, rejestr zleceń serwisowych, klasyfikacja zleceń, raporty kosztów serwisu
-----

Topic Tags:
Email: System floty – model przeglądów i alertów serwisowych
Topics: Fleet Management System, Service Management Model
-----
